In [57]:
from langchain_groq import ChatGroq
import os                                                                                                                                                                                                          
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.6,
    groq_api_key=api_key
)

response=llm.invoke("Where is heaven ?")
print(response.content)

The concept of heaven varies across cultures, religions, and belief systems, making it challenging to pinpoint a single location. Here are some perspectives:

1. **Religious views:**
	* Christianity: Heaven is often described as a spiritual realm, a place of eternal life, happiness, and union with God. Some Christian denominations believe it's a physical place, while others see it as a state of being.
	* Islam: Jannah (paradise) is the Islamic concept of heaven, a place of beauty, peace, and bliss, where believers will be rewarded for their good deeds.
	* Judaism: The afterlife is not a central concept in Judaism, but some texts describe a spiritual realm called Olam Ha-Ba (the world to come), where souls will be judged and either rewarded or punished.
	* Hinduism: Svarga (heaven) is a temporary realm where souls enjoy happiness and bliss before being reborn.
	* Buddhism: The concept of heaven is not central to Buddhism, but some texts describe a realm called Svarga (heaven) or Tusita 

In [58]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.cisco.com/jobs/IsAJob?projectId=1421663")
page_data=loader.load().pop().page_content
print(page_data)












Software Engineer – Network/Embedded/Application Development (Intern) - India UHR



 

















Skip to content




Cisco Careers


Menu




English
|
Français
Sign in






Search jobs
Careers home
Emerging Talent
Events
Stay In Touch













< Back to search results







Software Engineer – Network/Embedded/Application Development (Intern) - India UHR





Apply








Location:
Bangalore, India


Area of Interest
Engineer - Software


Job Type
Intern


Technology Interest
*None


Job Id
1421663







We are InnovatorsWe drive innovation to propel business transformation while maintaining operational quality.We are AcceleratorsWe accelerate digital solutions to generate cost savings and efficiency gains for enterprise growth and success.We are TransformersAs customer zero, we transform the customer experience by being our own customer first with agility, quality, and security, we continuously deliver business outcomes for our clients.What You’ll DoTeam Des

In [59]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={"page_data":page_data})
print(res.content)



```
[
  {
    "role": "Software Engineer – Network/Embedded/Application Development (Intern)",
    "experience": "Intern",
    "skills": "Computer science fundamentals, software engineering, TCP/IP networking fundamentals, programming and scripting languages like python, object-oriented design and programming, kernel programming, user space, system space, device drivers, BSPs, system level debugging techniques, creative problem-solving skills, excellent troubleshooting/debugging skills, excellent verbal and written skills",
    "description": "We are Innovators. We drive innovation to propel business transformation while maintaining operational quality. We are Accelerators. We accelerate digital solutions to generate cost savings and efficiency gains for enterprise growth and success. We are Transformers. As customer zero, we transform the customer experience by being our own customer first with agility, quality, and security, we continuously deliver business outcomes for our clients."

In [60]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer – Network/Embedded/Application Development (Intern)',
  'experience': 'Intern',
  'skills': 'Computer science fundamentals, software engineering, TCP/IP networking fundamentals, programming and scripting languages like python, object-oriented design and programming, kernel programming, user space, system space, device drivers, BSPs, system level debugging techniques, creative problem-solving skills, excellent troubleshooting/debugging skills, excellent verbal and written skills',
  'description': 'We are Innovators. We drive innovation to propel business transformation while maintaining operational quality. We are Accelerators. We accelerate digital solutions to generate cost savings and efficiency gains for enterprise growth and success. We are Transformers. As customer zero, we transform the customer experience by being our own customer first with agility, quality, and security, we continuously deliver business outcomes for our clients.'}]

In [61]:
import pandas as pd
df=pd.read_csv("portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [62]:
import chromadb
import uuid
client=chromadb.PersistentClient("vectorStore")
collection=client.get_or_create_collection(name="Portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])


In [63]:
links=collection.query(query_texts=["Experience in Python","Experience in React"],n_results=2).get("metadatas",[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [64]:
job=json_res
job

[{'role': 'Software Engineer – Network/Embedded/Application Development (Intern)',
  'experience': 'Intern',
  'skills': 'Computer science fundamentals, software engineering, TCP/IP networking fundamentals, programming and scripting languages like python, object-oriented design and programming, kernel programming, user space, system space, device drivers, BSPs, system level debugging techniques, creative problem-solving skills, excellent troubleshooting/debugging skills, excellent verbal and written skills',
  'description': 'We are Innovators. We drive innovation to propel business transformation while maintaining operational quality. We are Accelerators. We accelerate digital solutions to generate cost savings and efficiency gains for enterprise growth and success. We are Transformers. As customer zero, we transform the customer experience by being our own customer first with agility, quality, and security, we continuously deliver business outcomes for our clients.'}]

In [65]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Akshat, a software development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expertise in Software Development for Your Business Needs

Dear [Client],

I hope this email finds you well. My name is Akshat, and I am a software development executive at AtliQ, a leading AI & Software Consulting company. We came across your job posting for a Software Engineer – Network/Embedded/Application Development (Intern) and were impressed by your commitment to innovation and quality. 

At AtliQ, we specialize in facilitating the seamless integration of business processes through automated tools. Our team of experts has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We believe our capabilities align perfectly with your needs, and we would be delighted to discuss how we can contribute to your success.

Our expertise includes computer science fundamentals, software engineering, TCP/IP networking fundamentals, and proficiency in programming languages like Python. We a